# Multivariate Fuzzy C-medoids method: Implementation

## Equations

### $J= \sum_{i=1}^{c} \sum_{k=1}^{n} \sum_{j=1}^{p} \left(u_{ijk} \right)^{m} d_{ijk}$ - Objective function to minimize.

### $d_{ijk} = \left(x_{jk} - y_{ij} \right)^{2}$ - euclidian distance squared.

### $q = \argmin_{1 \le i \le c} \sum_{j=1}^p \sum_{k=1}^n (u_{ijk})^m \cdot d_{ijk}$ - prototype coordinate of a given cluster in feature j.

### $ u_{ijk} =  \left[\sum_{h=1}^{c}\sum_{l=1}^{p} \left(\frac{d_{ijk}}{d_{hlk}}\right)^{(1/(m-1))}  \right]^{-1} $ - membership degree of pattern k in cluster $C_{i}$ on the feature j.

### $\delta_{ik} = \sum_{j=1}^{p} u_{ijk}$ - represents an aggregation measure for all the p features.

## Constraints:

### - $u_{ijk} \in [0, 1]$ for all i, j and k;
### - $0 < \sum_{j=1}^{p} \sum_{k=1}^{n} u_{ijk} < n$ for all i and
### - $\sum_{i=1}^{c}\sum_{j=1}^{p}u_{ijk} = 1$ for all k.

## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score

## Tratamento dos dados

In [2]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

In [3]:
class MFCMedoids:
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m
        self.epsilon = 1e-10  # To prevent division by zero

    def initialize_u(self):
        return np.random.dirichlet(alpha=np.ones(self.c * self.p),
                                   size=self.n).reshape(self.n, self.c, self.p)

    def find_medoids(self, X, U):
        medoids = np.zeros((self.c, self.p))
        U_m = U ** self.m  # (n, c, p)

        # Para cada possível q (0 <= q < n), criamos um tensor de distâncias quadradas para todos os outros k e p
        # (n, n, p) -> distances_squared[k, q, j] = (X[k, j] - X[q, j]) ** 2
        distances_squared = np.abs(X[:, np.newaxis, :] - X[np.newaxis, :, :])  # city block

        for i in range(self.c):
            # Para o cluster i, obtemos U_m[:, i, :] -> shape (n, p)
            # Queremos calcular o custo de cada q ser o medoide: somatório sobre j e k de u_m[k, i, j] * d(k, q, j)
            
            # Expand u_m para fazer broadcast: (n, 1, p) para multiplicar com (n, n, p)
            u_m_expanded = U_m[:, i, :][:, np.newaxis, :]  # shape (n, 1, p)

            # Custo total para cada q: soma sobre k e j
            cost_per_q = np.sum(u_m_expanded * distances_squared, axis=(0, 2))  # shape (n,)

            best_q = np.argmin(cost_per_q)
            medoids[i] = X[best_q]

        return medoids

    def get_distances(self, X, medoids):
        return np.abs(X[:, np.newaxis, :] - medoids[np.newaxis, :, :]) # city block

    def update_u(self, D):
        D = np.maximum(D, self.epsilon)  # Avoid division by zero
        ratio = (D[:, np.newaxis, np.newaxis, :, :] / D[:, :, :, np.newaxis, np.newaxis]) ** (1 / (self.m - 1))
        return 1 / np.sum(ratio, axis=(3, 4))

    def get_objective_function(self, U, D):
        return np.sum((U ** self.m) * D)

def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-5):
    mfcm = MFCMedoids(c=num_clusters, X=dados, m=m)  # Create the MFCMedoids object

    U = mfcm.initialize_u()  # Initialize the membership matrix

    for _ in range(max_iter):
        medoids = mfcm.find_medoids(dados, U)
        D = mfcm.get_distances(dados, medoids)
        new_U = mfcm.update_u(D)
        
        # Check for convergence
        if np.linalg.norm(U - new_U) < epsilon:
            break
        
        U = new_U

    Delta = np.sum(U, axis=2)  # Summing over the second axis (variables j)

    return medoids, U, Delta

def monte_carlo_simulation(dados, labels, num_clusters, num_trials):
    results = []
    for _ in range(num_trials):
        print(_)
        medoids, U, Delta = mfcm_run(dados, num_clusters)
        predicted_labels = np.argmax(Delta, axis=1)
        ari = adjusted_rand_score(labels, predicted_labels)
        results.append(ari)
    mean_rand_index = np.mean(results)
    std_rand_index = np.std(results)
    return mean_rand_index, std_rand_index

In [ ]:
i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    if i == 5 or i == 6 or i == 7:
        num_clusters = 4
    else:
        num_clusters = 3
    df.drop("config", axis=1, inplace=True)
    if i == 5 or i == 6 or i == 7:
        df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
    else: 
        df["class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
    labels = df["class"].values
    df.drop("class", axis=1, inplace=True)
    dados = df.to_numpy()
    num_trials = 100
    mean_rand_index, std_rand_index = monte_carlo_simulation(dados, labels, num_clusters, num_trials)

    print(f"Monte Carlo MFCMdd Clustering Results for Config {i}")
    print(f"Mean Rand Index: {mean_rand_index:.4f}")
    print(f"Standard Deviation of Rand Index: {std_rand_index:.4f}")
    print("\n")
    i += 1

Os centroides das três classes estão localizados próximos uns dos outros:
$\mu_1 = \begin{bmatrix} 20 \\ 20 \end{bmatrix},$
$\mu_2 = \begin{bmatrix} 23 \\ 23 \end{bmatrix},$
$\mu_3 = \begin{bmatrix} 26 \\ 20 \end{bmatrix}$

As classes apresentam diferentes formas e orientações devido às suas matrizes de covariância:
$\Sigma_1 = \begin{bmatrix} 10 & 9 \\ 9 & 10 \end{bmatrix},$
$\Sigma_2 = \begin{bmatrix} 10 & -9 \\ -9 & 10 \end{bmatrix},$
$\Sigma_3 = \begin{bmatrix} 12 & 0 \\ 0 & 1 \end{bmatrix}$

- $\Sigma_1$ e $\Sigma_2$ geram distribuições elípticas com inclinação forte nas diagonais principais e secundárias, respectivamente.
- $\Sigma_3$ resulta em uma distribuição fortemente alongada no eixo $x$.

Cada classe possui $5\%$ de outliers, gerados a partir dos mesmos centros e covariâncias, mas com deslocamentos adicionais direcionados para regiões distantes dos centros originais. Os deslocamentos aplicados foram:
$\Delta_1 = \begin{bmatrix} -10 \\ 5 \end{bmatrix},$
$\Delta_2 = \begin{bmatrix} 10 \\ -10 \end{bmatrix},$
$\Delta_3 = \begin{bmatrix} 6 \\ 10 \end{bmatrix}$

In [ ]:
def config1_outliers(frac_outlier):
    np.random.seed(42)
    n = 150

    # Covariâncias exageradas para formas mais elípticas e inclinadas
    covs = [
        [[10, 9], [9, 10]],     # fortemente inclinado (diagonal)
        [[10, -9], [-9, 10]],   # diagonal oposta
        [[12, 0], [0, 1]]       # fortemente alongado no eixo x
    ]

    mus = [[20, 20], [23, 23], [26, 20]]  # centroides próximos!
    deslocamentos_outliers = [[-10, 5], [10, -10], [6, 10]]

    dados, rotulos, outlier_flags = [], [], []

    for i, (mu, cov, desloc) in enumerate(zip(mus, covs, deslocamentos_outliers)):
        classe = np.random.multivariate_normal(mu, cov, size=n)
        n_outliers = int(n * frac_outlier)

        for j, ponto in enumerate(classe):
            if j < n_outliers:
                outlier = ponto + desloc + np.random.normal(0, 1.8, size=2)
                dados.append(outlier)
                outlier_flags.append(1)
            else:
                dados.append(ponto)
                outlier_flags.append(0)
            rotulos.append(f'Classe {i+1}')

    # Garante que tudo fique no primeiro quadrante
    dados = np.array(dados)
    dados -= np.min(dados, axis=0)
    dados += 1

    df = pd.DataFrame(dados, columns=["x1", "x2"])
    df["Classe"] = rotulos
    df["Outlier"] = outlier_flags
    return df

In [ ]:
for frac_outlier in [0.05, 0.1, 0.15, 0.2, 0.25]:
    df = config1_outliers(frac_outlier)
    df.drop("Outlier", axis=1, inplace=True)
    df['Classe'].replace({'Classe 1': 0, 'Classe 2': 1, 'Classe 3': 2}, inplace=True)
    labels = df["Classe"].values
    df.drop("Classe", axis=1, inplace=True)
    dados = df.to_numpy()
    num_clusters = 3
    num_trials = 100
    media_indice_rand, dp_indice_rand = monte_carlo_simulation(dados, labels, num_clusters, num_trials)
    print("Fracao de outliers:", frac_outlier)
    print("Média do Índice Rand:", media_indice_rand)
    print("Desvio Padrão do Índice Rand:", dp_indice_rand)

In [ ]:
df.drop("Outlier", axis=1, inplace=True)
df['Classe'].replace({'Classe 1': 0, 'Classe 2': 1, 'Classe 3': 2}, inplace=True)
labels = df["Classe"].values
df.drop("Classe", axis=1, inplace=True)

In [ ]:
dados = df.to_numpy()
num_clusters = 3
num_trials = 100
media_indice_rand, dp_indice_rand = monte_carlo_simulation(dados, labels, num_clusters, num_trials)
print("Resultados de Monte Carlo para dados desafiadores (100 tentativas)")
print("Média do Índice Rand:", media_indice_rand)
print("Desvio Padrão do Índice Rand:", dp_indice_rand)